# Creating dummy values

Some time has passed since we created the database and now some customes have been (surprisingly) using the store's services and we have some data about them.


## Process

We're going to add dummy values to the `rental` and `customer` tables (like we did in the `store` table) so that we can de more complex queries in the final part of the project.

+ `customer`: using a couple libraries, we're going to add some random values to the table.
+ `rental`: we're going to use the original `rental.csv`, cleaning it and changing the date values so that it makes sense and randomnly changing the `customer_id` with the ids we've created on the previous steps

In [1]:
# Necessary installs

# %pip install faker

In [2]:
# libraries
import pandas as pd
pd.set_option('display.max_columns', None)

import numpy as np

import mysql.connector
from sqlalchemy import create_engine
from passwords import CURSOR

import warnings
warnings.filterwarnings('ignore')

import random as rd
from faker import Faker

import sys
sys.path.insert(1, '../')

from src import format_phone_number

## Filling customer table

In [3]:
# fetch the dable from sql
          
sql_query = pd.read_sql_query (
                                '''
                               SELECT
                               *
                               FROM customer
                               '''
                                , CURSOR)

df = pd.DataFrame(sql_query)

df

,customer_id,first_name,last_name,email,phone_number,gender,location


In [5]:
# we're going to create 25 dummy costumers

# set faker to get random spanish values
fake = Faker('es_ES')

# i + 1 will be the index
for i in range(25):
    # generate random gender
    gender = ['male', 'female', 'non-binary'][rd.randint(0,2)]
    
    # assign names depending on gender
    if gender == 'male':
        fname = fake.first_name_male()
        
    elif gender == 'female':
        fname = fake.first_name_female()
    
    else:
        fname = fake.first_name()
        
    lname = fake.last_name() + ' ' + fake.last_name()
        
    # creating dummy data with faker    
    location = fake.address()
    phone = fake.phone_number()
    email_domain = fake.domain_name()
    mail = f"{fname.lower()}.{lname.split()[0].lower()}@{email_domain}"
    
    # and append everything
    df.loc[i] = [i+1, fname, lname, mail, phone, gender, location]
    
    

In [6]:
df.head(10)

,customer_id,first_name,last_name,email,phone_number,gender,location
0,1,Carina,Puga Nicolau,carina.puga@belda-iglesia.com,+34 856422945,female,"Vial Augusto Palacio 49 Puerta 3 \nVizcaya, 37278"
1,2,Nazario,Moll Carlos,nazario.moll@vilar.org,+34928 81 71 83,male,"Calle de Federico Osuna 81 Puerta 1 \nOurense,..."
2,3,Osvaldo,Villalonga Gallo,osvaldo.villalonga@sanz.com,+34 875781066,male,Acceso de Vidal Torrijos 98\nSanta Cruz de Ten...
3,4,Faustino,Molina Luque,faustino.molina@uriarte-gallart.com,+34985 032 806,male,"C. de Sol Morante 85\nNavarra, 13633"
4,5,Carlito,Almazán Castañeda,carlito.almazán@cabo.org,+34885 027 911,male,"Pasaje de Federico Anglada 30\nÁvila, 13980"
5,6,Esperanza,Rivera Ferrando,esperanza.rivera@benet.com,+34 977 732 020,female,"Via de René Girón 216 Puerta 3 \nToledo, 30642"
6,7,Casemiro,Gallart Almazán,casemiro.gallart@pujol.es,+34815602095,male,"Rambla de Chelo Peñas 93 Puerta 5 \nÁlava, 18425"
7,8,Esteban,Lastra Marín,esteban.lastra@manrique.org,+34 826 965 403,male,"Alameda Amor Río 874 Piso 7 \nÁlava, 36857"
8,9,Cayetano,Fuentes Echeverría,cayetano.fuentes@alvarez.com,+34 873 753 515,non-binary,Urbanización de Godofredo Barrera 88 Apt. 31 \...
9,10,Glauco,Giralt Pérez,glauco.giralt@estevez.es,+34 985 34 06 01,male,"Cañada de Adriana Ibañez 47\nZamora, 03005"


In [7]:
help(format_phone_number)

Help on function format_phone_number in module src:

format_phone_number(phone)
    Returns the phone number introduced to the standar Spanish format "+34 XXX XXX XXX".
    
    Arguments:
        phone: a string of a phone number with or without '+' and with or without 34 at the beggining (it will be added if it doesn't have it).
                
                If the string is not digit, the function will not register it as an error and will return unexpected values.
        
    Returns:
        formatted_phone: a string with the formatted phone like this "+34 XXX XXX XXX"



In [8]:
# Apply the function to the 'Phone' column
df.phone_number = df.phone_number.apply(format_phone_number)

df.head()

,customer_id,first_name,last_name,email,phone_number,gender,location
0,1,Carina,Puga Nicolau,carina.puga@belda-iglesia.com,+34 564 229 45,female,"Vial Augusto Palacio 49 Puerta 3 \nVizcaya, 37278"
1,2,Nazario,Moll Carlos,nazario.moll@vilar.org,+34 288 171 83,male,"Calle de Federico Osuna 81 Puerta 1 \nOurense,..."
2,3,Osvaldo,Villalonga Gallo,osvaldo.villalonga@sanz.com,+34 757 810 66,male,Acceso de Vidal Torrijos 98\nSanta Cruz de Ten...
3,4,Faustino,Molina Luque,faustino.molina@uriarte-gallart.com,+34 850 328 06,male,"C. de Sol Morante 85\nNavarra, 13633"
4,5,Carlito,Almazán Castañeda,carlito.almazán@cabo.org,+34 850 279 11,male,"Pasaje de Federico Anglada 30\nÁvila, 13980"


In [9]:
# and now we append all the dummy data
df.to_sql(name='customer',
          con=CURSOR,
          if_exists='append',
          index=False)

25

## Filling rental table

In [10]:
# import old rental table
df = pd.read_csv('../src/rental.csv')

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   rental_id     1000 non-null   int64 
 1   rental_date   1000 non-null   object
 2   inventory_id  1000 non-null   int64 
 3   customer_id   1000 non-null   int64 
 4   return_date   1000 non-null   object
 5   staff_id      1000 non-null   int64 
 6   last_update   1000 non-null   object
dtypes: int64(4), object(3)
memory usage: 54.8+ KB


`last_update` was determined to be a useless column, so we're going to drop it along `staff_id` since Deli Ushional store is a self-service automatic store without staff (realisticly there would be guards, storage staff and so on but for the sake of simplicity that is going to be ommited).

In [12]:
df.drop(['last_update', 'staff_id'], axis=1, inplace=True)

In [13]:
# we're going to change customer_id to the values we created before 

df.customer_id = df.customer_id.apply(lambda x: rd.randint(1, 25))

df.customer_id.unique()

array([19, 10, 18,  6, 25, 24,  1, 11,  5,  3, 17,  7, 21,  9, 16, 12,  2,
       23,  8, 15, 14,  4, 20, 22, 13], dtype=int64)

In [15]:
df.rental_date = df.rental_date.apply(lambda x: x.replace(x[:4], '2023'))

df.return_date = df.return_date.apply(lambda x: x.replace(x[:4], '2023'))

df.head()

,rental_id,rental_date,inventory_id,customer_id,return_date
0,1,2023-05-24 22:53:30,367,19,2023-05-26 22:04:30
1,2,2023-05-24 22:54:33,1525,10,2023-05-28 19:40:33
2,3,2023-05-24 23:03:39,1711,18,2023-06-01 22:12:39
3,4,2023-05-24 23:04:41,2452,6,2023-06-03 01:43:41
4,5,2023-05-24 23:05:21,2079,25,2023-06-02 04:33:21


In [16]:
# finally, inventory_id has numbers higher than the ones we have in inventory. Since this is dummy data, we're going to assign random numbers to the inventory_id using the ids we have

df.inventory_id = df.inventory_id.apply(lambda x: rd.randint(1, 1000))

Now that the `rental` table has been modified to support our needs, it's time to export it to sql.

In [17]:
# first we downcast the ints

for c in df.select_dtypes('integer'):
    df[c] = pd.to_numeric(df[c], downcast='integer')
    
# and now we append to the db
df.to_sql(name='rental',
          con=CURSOR,
          if_exists='append',
          index=False)

1000